## 카페 추천 챗봇 

In [53]:
import os
from openai import AzureOpenAI
import json
from IPython.display import Image as IPyImage, display
import requests
from PIL import Image
from io import BytesIO

def request_SDK(prompt, chat_history):
    try:
        endpoint = os.getenv("ENDPOINT_URL", "https://7ai015-openai.openai.azure.com/")
        deployment = os.getenv("DEPLOYMENT_NAME", "7ai015-gpt-4o-mini")
        search_endpoint = os.getenv("SEARCH_ENDPOINT", "https://7ai015-aisearch.search.windows.net")
        search_key = os.getenv("SEARCH_KEY", "QkjHtmZVcwnXNZDgY8uH46TkT5N3OkXauMa1OH0le2AzSeDXapGA")
        search_index = os.getenv("SEARCH_INDEX_NAME", "pet-index")
        subscription_key = os.getenv("AZURE_OPENAI_API_KEY", "937jKABfHgomOxXzVz4q7cxqEFwjMqrZ1HxIr4nUs4SfYQqD2SQrJQQJ99BFACfhMk5XJ3w3AAABACOG8ETF")
        search_semantic = "pet-index-semantic-configuration"

       
        client = AzureOpenAI(
            azure_endpoint=endpoint,
            api_key=subscription_key,
            api_version="2025-01-01-preview",
        )

        chat_prompt = [
            {
                "role": "system",
                "content": "너는 반려동물 동반 카페를 추천해주는 AI 어시스턴트야. 사용자가 원하는 정보를 제공해주면 돼."
            }
        ]

        for msg in chat_history:
            chat_prompt.append({
                "role": msg['role'],
                "content": msg['content']
            })

        chat_prompt.append({"role": "user", "content": prompt})

        completion = client.chat.completions.create(
            model=deployment,
            messages=chat_prompt,
            max_tokens=800,
            temperature=0.5,
            top_p=0.95,
            extra_body={
            "data_sources": [{
                "type": "azure_search",
                "parameters": {
                    "endpoint": f"{search_endpoint}",
                    "index_name": f"{search_index}",
                    "semantic_configuration": search_semantic,
                    "query_type": "semantic",
                    "strictness": 2,
                    "top_n_documents": 3,
                    "authentication": {
                    "type": "api_key",
                    "key": f"{search_key}"
                    }
                }
                }]
            }
        )
        response = completion.choices[0].message.content
        
        chat_prompt.append({"role": "assistant", "content": response})

        if len(chat_prompt) > 11:
            chat_prompt = chat_prompt[0] + chat_prompt[-10:]
            
        # citations 출력 
        response2 = ""
        citations = completion.choices[0].message.context["citations"]
        for i in range(len(citations)):
            response2 += f"\n--- {i+1}번째 출처 ---\n{citations[i]['content']}\n"
        
        return response, response2

    except Exception as ex:
        return ex
    
def dalle(prompt):

    # 이러한 환경 변수를 설정하거나 다음 값을 편집해야 합니다.
    endpoint = os.getenv("AZURE_OPENAI_ENDPOINT", "https://7ai004-openai.openai.azure.com/")
    api_version = os.getenv("OPENAI_API_VERSION", "2024-04-01-preview")
    deployment = os.getenv("DEPLOYMENT_NAME", "dall-e-3")
    api_key = os.getenv("AZURE_OPENAI_API_KEY", "ECVaBECoiTeZnvco7KeQmbNFHichkEwF9Pxf7qQWy4BxMlCLCc2rJQQJ99BFACfhMk5XJ3w3AAABACOGtTWb")

    client = AzureOpenAI(
        api_version=api_version,
        azure_endpoint=endpoint,
        api_key=api_key,
    )

    result = client.images.generate(
        model=deployment,
        prompt=prompt,
        n=1,
        style="vivid",
        quality="standard",
    )

    image_url = json.loads(result.model_dump_json())['data'][0]['url']

    # 결과창에 이미지 표시
    # display(IPyImage(url=image_url))

    # 이미지 파일에 저장
    response = requests.get(image_url)
    img = Image.open(BytesIO(response.content))

    file_path = os.path.join("dalle3_images", f'{prompt}.jpg')
    img.save(file_path)

    return image_url

def tts(chat_history):
    if not chat_history or not isinstance(chat_history, list):
        return None

    last_message = ""
    # 가장 마지막 assistant 메시지를 찾아 TTS로 사용
    for msg in reversed(chat_history):
        if msg['role'] == 'assistant':
            last_message = msg['content']
            break

    if not last_message:
        return None

    client = AzureOpenAI(
        azure_endpoint="https://7ai006-openai.openai.azure.com/",
        api_key="4ZoLnu5PUD3SjieeK5K4iIBovWIaa6yjoC26NIw9LtUxnmNjDEsKJQQJ99BFACfhMk5XJ3w3AAABACOGuwPz",  # 자신의 API 키로 바꾸세요
        api_version="2025-03-01-preview"
    )

    response = client.audio.speech.create(
        model="tts",
        voice="alloy",
        input=last_message  # 문자열만 전달
    )

    file_path = "tts_output.mp3"
    response.write_to_file(file_path)

    return file_path


In [54]:
import gradio as gr
import html
import IPython.display as ipd

def chat(user_input, chat_history):
    try:
        bot_response, citations = request_SDK(user_input, chat_history)
        full_response = bot_response + "\n\n📌 출처:\n" + citations

        chat_history.append({'role': 'user', 'content': user_input})
        chat_history.append({'role': 'assistant', 'content': bot_response})

    except Exception as e:
        full_response = "응답이 없습니다."
    return "", chat_history

def generate_image(image):
    result = dalle(image)
    return result

with gr.Blocks() as demo:
    gr.Markdown("## 🐶🐱")

    with gr.Tabs() :
        with gr.Tab('🐾챗봇'):
            gr.Markdown("## 🐾반려동물 동반 카페 추천 챗봇")
            chatbot = gr.Chatbot(label="챗봇", type='messages', height=600)
            user_input = gr.Textbox(label="질문을 입력하세요.")
            submit_btn = gr.Button("전송")
            tts_btn = gr.Button("음성 생성")
            audio_output = gr.Audio(label="생성된 음성", type="filepath")

            submit_btn.click(fn=chat, inputs=[user_input, chatbot], outputs=[user_input, chatbot])
            user_input.submit(fn=chat, inputs= [user_input, chatbot], outputs= [user_input, chatbot])
            tts_btn.click(fn=tts, inputs=chatbot, outputs=audio_output)

        with gr.Tab('🎨이미지 생성'):
            gr.Markdown("## 🎨원하는 카페 분위기를 그려보자!")
            input_textbox = gr.Textbox(label='프롬프트를 입력하세요.')
            submit_button = gr.Button('이미지 생성') 
            output_image = gr.Image(label="생성된 이미지", height=400)
            
            submit_button.click(fn=generate_image, inputs=[input_textbox], outputs=[output_image])

demo.launch()

* Running on local URL:  http://127.0.0.1:7925
* To create a public link, set `share=True` in `launch()`.


In [ ]:
import os
from openai import AzureOpenAI
import json
from IPython.display import Image as IPyImage, display
import requests
from PIL import Image
from io import BytesIO

def request_SDK(prompt, chat_history):
    try:
        endpoint = os.getenv("ENDPOINT_URL", "https://7ai015-openai.openai.azure.com/")
        deployment = os.getenv("DEPLOYMENT_NAME", "7ai015-gpt-4o-mini")
        search_endpoint = os.getenv("SEARCH_ENDPOINT", "https://7ai015-aisearch.search.windows.net")
        search_key = os.getenv("SEARCH_KEY", "QkjHtmZVcwnXNZDgY8uH46TkT5N3OkXauMa1OH0le2AzSeDXapGA")
        search_index = os.getenv("SEARCH_INDEX_NAME", "pet-index")
        subscription_key = os.getenv("AZURE_OPENAI_API_KEY", "937jKABfHgomOxXzVz4q7cxqEFwjMqrZ1HxIr4nUs4SfYQqD2SQrJQQJ99BFACfhMk5XJ3w3AAABACOG8ETF")
        search_semantic = "pet-index-semantic-configuration"

       
        client = AzureOpenAI(
            azure_endpoint=endpoint,
            api_key=subscription_key,
            api_version="2025-01-01-preview",
        )

        chat_prompt = [
            {
                "role": "system",
                "content": "너는 반려동물 동반 카페를 추천해주는 AI 어시스턴트야. 사용자가 원하는 정보를 제공해주면 돼."
            }
        ]

        for msg in chat_history:
            chat_prompt.append({
                "role": msg['role'],
                "content": msg['content']
            })

        chat_prompt.append({"role": "user", "content": prompt})

        completion = client.chat.completions.create(
            model=deployment,
            messages=chat_prompt,
            max_tokens=800,
            temperature=0.5,
            top_p=0.95,
            extra_body={
            "data_sources": [{
                "type": "azure_search",
                "parameters": {
                    "endpoint": f"{search_endpoint}",
                    "index_name": f"{search_index}",
                    "semantic_configuration": search_semantic,
                    "query_type": "semantic",
                    "strictness": 2,
                    "top_n_documents": 3,
                    "authentication": {
                    "type": "api_key",
                    "key": f"{search_key}"
                    }
                }
                }]
            }
        )
        response = completion.choices[0].message.content
        
        chat_prompt.append({"role": "assistant", "content": response})

        if len(chat_prompt) > 11:
            chat_prompt = chat_prompt[0] + chat_prompt[-10:]
            
        # citations 출력 
        response2 = ""
        citations = completion.choices[0].message.context["citations"]
        for i in range(len(citations)):
            response2 += f"\n--- {i+1}번째 출처 ---\n{citations[i]['content']}\n"
        
        return response, response2

    except Exception as ex:
        return ex
    
def dalle(prompt):

    # 이러한 환경 변수를 설정하거나 다음 값을 편집해야 합니다.
    endpoint = os.getenv("AZURE_OPENAI_ENDPOINT", "https://7ai004-openai.openai.azure.com/")
    api_version = os.getenv("OPENAI_API_VERSION", "2024-04-01-preview")
    deployment = os.getenv("DEPLOYMENT_NAME", "dall-e-3")
    api_key = os.getenv("AZURE_OPENAI_API_KEY", "ECVaBECoiTeZnvco7KeQmbNFHichkEwF9Pxf7qQWy4BxMlCLCc2rJQQJ99BFACfhMk5XJ3w3AAABACOGtTWb")

    client = AzureOpenAI(
        api_version=api_version,
        azure_endpoint=endpoint,
        api_key=api_key,
    )

    result = client.images.generate(
        model=deployment,
        prompt=prompt,
        n=1,
        style="vivid",
        quality="standard",
    )

    image_url = json.loads(result.model_dump_json())['data'][0]['url']

    # 결과창에 이미지 표시
    # display(IPyImage(url=image_url))

    # 이미지 파일에 저장
    response = requests.get(image_url)
    img = Image.open(BytesIO(response.content))

    file_path = os.path.join("dalle3_images", f'{prompt}.jpg')
    img.save(file_path)

    return image_url

def tts():
    client = AzureOpenAI(
        azure_endpoint="https://7ai006-openai.openai.azure.com/",
        api_key="4ZoLnu5PUD3SjieeK5K4iIBovWIaa6yjoC26NIw9LtUxnmNjDEsKJQQJ99BFACfhMk5XJ3w3AAABACOGuwPz",
        api_version="2025-03-01-preview",
    )

    response = client.audio.speech.create(
        model = 'tts',
        voice = 'alloy',
        input = response
    )

    response.write_to_file('tts_test.mp3')


In [48]:
import gradio as gr
import html
import IPython.display as ipd

def chat(user_input, chat_history):
    try:
        bot_response, citations = request_SDK(user_input, chat_history)
        full_response = bot_response + "\n\n📌 출처:\n" + citations

        chat_history.append({'role': 'user', 'content': user_input})
        chat_history.append({'role': 'assistant', 'content': full_response})

    except Exception as e:
        full_response = "응답이 없습니다."
    return "", chat_history

def generate_image(image):
    result = dalle(image)
    return result

with gr.Blocks() as demo:
    gr.Markdown("## 🐶🐱")

    with gr.Tabs() :
        with gr.Tab('🐾챗봇'):
            gr.Markdown("## 🐾반려동물 동반 카페 추천 챗봇")
            chatbot = gr.Chatbot(label="챗봇", type='messages', height=600)
            user_input = gr.Textbox(label="질문을 입력하세요.")
            submit_btn = gr.Button("전송")

            submit_btn.click(fn=chat, inputs=[user_input, chatbot], outputs=[user_input, chatbot])
            user_input.submit(fn=chat, inputs= [user_input, chatbot], outputs= [user_input, chatbot])

        with gr.Tab('🎨이미지 생성'):
            gr.Markdown("## 🎨원하는 카페 분위기를 그려보자!")
            input_textbox = gr.Textbox(label='프롬프트를 입력하세요.')
            submit_button = gr.Button('이미지 생성') 
            output_image = gr.Image(label="생성된 이미지", height=400)
            
            submit_button.click(fn=generate_image, inputs=[input_textbox], outputs=[output_image])

demo.launch()

* Running on local URL:  http://127.0.0.1:7923
* To create a public link, set `share=True` in `launch()`.
